# fastai Efficientnet B3 with Ranger

This kernel will train a model that can achieve a score of ~83.5%

In [ ]:
from fastai.vision.all import *
set_seed(999)

We're going to use the `wwf` library with `timm` integrated

> Note: this is a training *only* kernel

In [ ]:
!pip install wwf timm -qqq

In [ ]:
from wwf.vision.timm import *

We'll read in our data similar to the original kernel I made [here](https://www.kaggle.com/muellerzr/cassava-fastai-starter)

In [ ]:
df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
idx2lbl = {0:"Cassava Bacterial Blight (CBB)",
          1:"Cassava Brown Streak Disease (CBSD)",
          2:"Cassava Green Mottle (CGM)",
          3:"Cassava Mosaic Disease (CMD)",
          4:"Healthy"}

df['label'].replace(idx2lbl, inplace=True)

In [ ]:
data_path = Path('../input/cassava-leaf-disease-classification/')

In [ ]:
blocks = (ImageBlock, CategoryBlock)
splitter = RandomSplitter(0.2, seed=999)
def get_x(row): return data_path/'train_images'/row['image_id']
def get_y(row): return row['label']
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]

In [ ]:
block = DataBlock(blocks = blocks,
                 get_x = get_x,
                 get_y = get_y,
                 splitter = splitter,
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms)

In [ ]:
dls = block.dataloaders(df, bs=32)

Next we'll train our model:

In [ ]:
learn = timm_learner(dls, 'efficientnet_b3', loss_func=LabelSmoothingCrossEntropy(), metrics=[accuracy], opt_func=ranger)
learn.to_native_fp16()

In [ ]:
learn.lr_find()

1e-2 works fairly well here, in the future I want to play with weight-decay some to see if it can improve the performance. We'll train with Gradient Accumulation, MixUp, and ReduceLROnPlateu

In [ ]:
learn.fit_flat_cos(1,1e-2, cbs=[GradientAccumulation(), MixUp(), ReduceLROnPlateau()])

In [ ]:
learn.recorder.plot_loss()

We'll unfreeze and see how the lr plot looks

In [ ]:
learn.unfreeze()
learn.lr_find()

Based on emprical work I've done, 1e-3 will be stable enough for what we want. While the metric does get a bit funky here notice the train and valid loss mostly just keep going down together

In [ ]:
learn.fit_flat_cos(10,1e-3, cbs=[GradientAccumulation(), MixUp(), ReduceLROnPlateau()])

In [ ]:
learn.to_native_fp32()

Finally you can save and export your model

In [ ]:
learn.save('b3')

## Inference

This should be adapted in a seperate kernel, but here is a general guideline for submitting your code I have been doing:


1. Create a dataset with your saved model
2. Follow [this](https://www.kaggle.com/muellerzr/submission-notebook) notebook for generating your submission.